In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/seattle/calendar.csv
/kaggle/input/seattle/listings.csv
/kaggle/input/seattle/reviews.csv


In [2]:
import re
from collections import Counter

In [3]:
seattle_calendar = pd.read_csv("/kaggle/input/seattle/calendar.csv")
seattle_listings = pd.read_csv("/kaggle/input/seattle/listings.csv")
seattle_reviews = pd.read_csv("/kaggle/input/seattle/reviews.csv")

Just taking a quick look at what the dataframes look like

In [4]:
seattle_calendar.head()

,listing_id,date,available,price
0,241032,2016-01-04,t,$85.00
1,241032,2016-01-05,t,$85.00
2,241032,2016-01-06,f,NaN
3,241032,2016-01-07,f,NaN
4,241032,2016-01-08,f,NaN


In [5]:
seattle_calendar.describe()

,listing_id
count,1.393570e+06
mean,5.550111e+06
std,2.962274e+06
min,3.335000e+03
25%,3.258213e+06
50%,6.118244e+06
75%,8.035212e+06
max,1.034016e+07


In [6]:
seattle_listings.head()

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pan

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,241032,https://www.airbnb.com/rooms/241032,20160104002432,2016-01-04,Stylish Queen Anne Apartment,NaN,Make your self at home in this charming one-be...,Make your self at home in this charming one-be...,none,NaN,...,10.0,f,NaN,WASHINGTON,f,moderate,f,f,2,4.07
1,953595,https://www.airbnb.com/rooms/953595,20160104002432,2016-01-04,Bright & Airy Queen Anne Apartment,Chemically sensitive? We've removed the irrita...,"Beautiful, hypoallergenic apartment in an extr...",Chemically sensitive? We've removed the irrita...,none,"Queen Anne is a wonderful, truly functional vi...",...,10.0,f,NaN,WASHINGTON,f,strict,t,t,6,1.48
2,3308979,https://www.airbnb.com/rooms/3308979,20160104002432,2016-01-04,New Modern House-Amazing water view,New modern house built in 2013. Spectacular s...,"Our house is modern, light and fresh with a wa...",New modern house built in 2013. Spectacular s...,none,Upper Queen Anne is a charming neighborhood fu...,...,10.0,f,NaN,WASHINGTON,f,strict,f,f,2,1.15
3,7421966,https://www.airbnb.com/rooms/7421966,20160104002432,2016-01-04,Queen Anne Chateau,A charming apartment that sits atop Queen Anne...,NaN,A charming apartment that sits atop Queen Anne...,none,NaN,...,NaN,f,NaN,WASHINGTON,f,flexible,f,f,1,NaN
4,278830,https://www.airbnb.com/rooms/278830,20160104002432,2016-01-04,Charming craftsman 3 bdm house,Cozy family craftman house in beautiful neighb...,Cozy family craftman house in beautiful neighb...,Cozy family craftman house in beautiful neighb...,none,We are in the beautiful neighborhood of Queen ...,...,9.0,f,NaN,WASHINGTON,f,strict,f,f,1,0.89


In [7]:
seattle_listings.describe()

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pan

,id,scrape_id,host_id,host_listings_count,host_total_listings_count,latitude,longitude,accommodates,bathrooms,bedrooms,...,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,calculated_host_listings_count,reviews_per_month
count,3.818000e+03,3.818000e+03,3.818000e+03,3816.000000,3816.000000,3818.000000,3818.000000,3818.000000,3802.000000,3812.000000,...,3171.000000,3160.000000,3165.000000,3160.000000,3167.000000,3163.000000,3162.000000,0.0,3818.000000,3191.000000
mean,5.550111e+06,2.016010e+13,1.578556e+07,7.157757,7.157757,47.628961,-122.333103,3.349398,1.259469,1.307712,...,94.539262,9.636392,9.556398,9.786709,9.809599,9.608916,9.452245,NaN,2.946307,2.078919
std,2.962660e+06,0.000000e+00,1.458382e+07,28.628149,28.628149,0.043052,0.031745,1.977599,0.590369,0.883395,...,6.606083,0.698031,0.797274,0.595499,0.568211,0.629053,0.750259,NaN,5.893029,1.822348
min,3.335000e+03,2.016010e+13,4.193000e+03,1.000000,1.000000,47.505088,-122.417219,1.000000,0.000000,0.000000,...,20.000000,2.000000,3.000000,2.000000,2.000000,4.000000,2.000000,NaN,1.000000,0.020000
25%,3.258256e+06,2.016010e+13,3.275204e+06,1.000000,1.000000,47.609418,-122.354320,2.000000,1.000000,1.000000,...,93.000000,9.000000,9.000000,10.000000,10.000000,9.000000,9.000000,NaN,1.000000,0.695000
50%,6.118244e+06,2.016010e+13,1.055814e+07,1.000000,1.000000,47.623601,-122.328874,3.000000,1.000000,1.000000,...,96.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,NaN,1.000000,1.540000
75%,8.035127e+06,2.016010e+13,2.590309e+07,3.000000,3.000000,47.662694,-122.310800,4.000000,1.000000,2.000000,...,99.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,NaN,2.000000,3.000000
max,1.034016e+07,2.016010e+13,5.320861e+07,502.000000,502.000000,47.733358,-122.240607,16.000000,8.000000,7.000000,...,100.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,NaN,37.000000,12.150000


In [8]:
seattle_reviews.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,7202016,38917982,2015-07-19,28943674,Bianca,Cute and cozy place. Perfect location to every...
1,7202016,39087409,2015-07-20,32440555,Frank,Kelly has a great room in a very central locat...
2,7202016,39820030,2015-07-26,37722850,Ian,"Very spacious apartment, and in a great neighb..."
3,7202016,40813543,2015-08-02,33671805,George,Close to Seattle Center and all it has to offe...
4,7202016,41986501,2015-08-10,34959538,Ming,Kelly was a great host and very accommodating ...


In [9]:
seattle_reviews.describe()

,listing_id,id,reviewer_id
count,8.484900e+04,8.484900e+04,8.484900e+04
mean,3.005067e+06,3.058765e+07,1.701301e+07
std,2.472877e+06,1.636613e+07,1.353704e+07
min,4.291000e+03,3.721000e+03,1.500000e+01
25%,7.946330e+05,1.725127e+07,5.053141e+06
50%,2.488228e+06,3.228809e+07,1.413476e+07
75%,4.694479e+06,4.457648e+07,2.762402e+07
max,1.024814e+07,5.873651e+07,5.281274e+07


In [10]:
seattle_calendar.columns.values

array(['listing_id', 'date', 'available', 'price'], dtype=object)

In [11]:
seattle_listings.columns.values

array(['id', 'listing_url', 'scrape_id', 'last_scraped', 'name',
       'summary', 'space', 'description', 'experiences_offered',
       'neighborhood_overview', 'notes', 'transit', 'thumbnail_url',
       'medium_url', 'picture_url', 'xl_picture_url', 'host_id',
       'host_url', 'host_name', 'host_since', 'host_location',
       'host_about', 'host_response_time', 'host_response_rate',
       'host_acceptance_rate', 'host_is_superhost', 'host_thumbnail_url',
       'host_picture_url', 'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'street',
       'neighbourhood', 'neighbourhood_cleansed',
       'neighbourhood_group_cleansed', 'city', 'state', 'zipcode',
       'market', 'smart_location', 'country_code', 'country', 'latitude',
       'longitude', 'is_location_exact', 'property_type', 'room_type',
       'accommodates', 'bathrooms', 'bedrooms', 'beds', 'bed_type',
       

In [12]:
seattle_reviews.columns.values

array(['listing_id', 'id', 'date', 'reviewer_id', 'reviewer_name',
       'comments'], dtype=object)

In [13]:
#figuring out what each of the features in seattle_listings means
for feature in seattle_listings.columns.values:
    print(seattle_listings[feature].head())
    print("")

0     241032
1     953595
2    3308979
3    7421966
4     278830
Name: id, dtype: int64

0     https://www.airbnb.com/rooms/241032
1     https://www.airbnb.com/rooms/953595
2    https://www.airbnb.com/rooms/3308979
3    https://www.airbnb.com/rooms/7421966
4     https://www.airbnb.com/rooms/278830
Name: listing_url, dtype: object

0    20160104002432
1    20160104002432
2    20160104002432
3    20160104002432
4    20160104002432
Name: scrape_id, dtype: int64

0    2016-01-04
1    2016-01-04
2    2016-01-04
3    2016-01-04
4    2016-01-04
Name: last_scraped, dtype: object

0           Stylish Queen Anne Apartment
1     Bright & Airy Queen Anne Apartment
2    New Modern House-Amazing water view
3                     Queen Anne Chateau
4         Charming craftsman 3 bdm house
Name: name, dtype: object

0                                                  NaN
1    Chemically sensitive? We've removed the irrita...
2    New modern house built in 2013.  Spectacular s...
3    A charming apartmen

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


It might be possible to make some simple categorical features out of the text columns, such as whether certain words appear in the description or review for example, so I will keep those for now. Selected features that could be relevant for predicting the label (price)

In [14]:
#keeping features that may either be of use in determining duplicates or for the
#actual price prediction
selected_listing_columns = ["last_review", "street", "name", "summary", "space", "description", 
                            "neighborhood_overview", "notes", "transit", "host_since", 
                           "host_about", "host_response_time", "host_response_rate", 
                           "host_is_superhost", "host_neighbourhood", "host_listings_count", 
                           "host_total_listings_count", "host_has_profile_pic", 
                           "host_identity_verified", "neighbourhood", "neighbourhood_cleansed", 
                           "neighbourhood_group_cleansed", "property_type", "room_type",
                           "accommodates", "bathrooms", "bedrooms", "beds", "bed_type",
                           "amenities", "square_feet", "review_scores_rating", 
                           "review_scores_accuracy", "review_scores_cleanliness", 
                           "review_scores_checkin", "review_scores_communication", 
                           "review_scores_location", "review_scores_value", "cancellation_policy", 
                           "calculated_host_listings_count", "price"]
new_seattle_listings = seattle_listings[selected_listing_columns].copy()
print("The new seattle listings dataframe will use", 
      new_seattle_listings.shape[1] - 1, "features, and the output label will be", 
     new_seattle_listings.columns[-1])

The new seattle listings dataframe will use 40 features, and the output label will be price


In [15]:
print(new_seattle_listings.shape)
print(new_seattle_listings['notes'][9])
print("")
print(new_seattle_listings['notes'][1])
print("")
print(new_seattle_listings['price'][1])
print("")
print(new_seattle_listings['price'][9])
print(new_seattle_listings['notes'][1] == new_seattle_listings['notes'][9])
print(new_seattle_listings['price'][1] == new_seattle_listings['price'][9])

(3818, 41)
What's up with the free pillows?  Our home was renovated and restored with health & safety in mind, especially the elimination/reduction of indoor allergens, toxins & other pollutants.  (For this reason, we have no carpets, just hardwood & slate floors, with radiant heat.)  Pillows tend to house a LOT of allergens over time, even when just used by a single owner.  The statistics make many people feel pretty queasy but, essentially, pillows become great breeding grounds for bacteria and dust mites, not to mention drool, hair oils and perspiration. Since most people sweat more than a pint a night, the pillow can also easily trap mildew and mold. We use dust mite protectors for mattresses and duvets, but haven't found any that work well for pillows that maintain comfort/elasticity without the use of outgassing materials.  So, I let the guests take their pillows but ask that they leave the pillowcases. And if you leave the pillows behind, they'll be donated to a local women's sh

Look at the entries above! It appears that there are duplicate entries in the dataframe. We want to remove these duplicates so that every entry is unique.

In [16]:
#Remove duplicate entries. Here we are just removing exact duplicates, which could be a
#limitation if there are near duplicates but I am just handling this for now
print("Shape of original dataframe (with features selected):",new_seattle_listings.shape)
exact_duplicates_dropped = new_seattle_listings.drop_duplicates()
print("Shape after dropping exact duplicates:",exact_duplicates_dropped.shape)
subset_dropped_listings_np = new_seattle_listings.drop_duplicates(subset=['notes', 'price'])
print("Shape after dropping duplicates with exact match of notes and price:",
      subset_dropped_listings_np.shape)
subset_dropped_listings_notes = new_seattle_listings.drop_duplicates(subset=['notes'])
print("Shape after dropping duplicates with exact match of notes:",
      subset_dropped_listings_notes.shape)

#trying a subset of house-related stuff, maybe this will reveal duplicates accurately?
#not sure 
subset_dropped_listings_house_features = new_seattle_listings.drop_duplicates(
    subset=['property_type', 'bathrooms', 'bedrooms', 'square_feet', 'neighbourhood'])
print("Shape after dropping duplicates with matching house related features:", 
     subset_dropped_listings_house_features.shape)
subset_dropped_listings_name = new_seattle_listings.drop_duplicates(
    subset=['name']
)
print("Shape after dropping duplicates with exact match of name: ",
     subset_dropped_listings_name.shape)

Shape of original dataframe (with features selected): (3818, 41)
Shape after dropping exact duplicates: (3818, 41)
Shape after dropping duplicates with exact match of notes and price: (2337, 41)
Shape after dropping duplicates with exact match of notes: (2000, 41)
Shape after dropping duplicates with matching house related features: (1071, 41)
Shape after dropping duplicates with exact match of name:  (3792, 41)


As we can see from some experimentation dropping duplicates, there are no exact duplicates! However, if we limit to dropping if the exact price and notes are the same, we get a significant reduction in entries. We reduce even more entries if we just look for if the notes are the same. This suggestions, for example, that someone may have put the same listing up but with different prices, perhaps due to increased demand or they weren't able to get anyone to book so they decreased the price. Also, it's possible that some entries are duplicates but won't be caught using notes and price as the subset, but will be caught using a different subset. We cannot try all the subsets of the features that we have, so we will need an effective method to identify duplicates entries.

In [17]:
#I need to see what the full listings look like for some of these examples that have a 
#couple duplicate features, for example notes and price, look like. We know indices 1 and 9
#are potential duplicates

#print(new_seattle_listings.iloc[1])
print("")
#print(new_seattle_listings.iloc[9])
unequal_features = []
for feature in new_seattle_listings:
    equal_feature = (new_seattle_listings[feature][1] == new_seattle_listings[feature][9])
    print(feature, equal_feature)
    if not equal_feature:
        unequal_features.append(feature)
print("----------------")

#these two neighborhood_overview look exactly the same so printing out length to see why they
#are unequal
print(len(new_seattle_listings["neighborhood_overview"][1]))
print(len(new_seattle_listings["neighborhood_overview"][9]))
for f in unequal_features:
    print(f, "for listing 1:", new_seattle_listings[f][1])
    print("")
    print(f, "for listing 9:", new_seattle_listings[f][9])
    print("")


last_review False
street True
name False
summary False
space False
description False
neighborhood_overview False
notes True
transit True
host_since True
host_about True
host_response_time True
host_response_rate True
host_is_superhost True
host_neighbourhood True
host_listings_count True
host_total_listings_count True
host_has_profile_pic True
host_identity_verified True
neighbourhood True
neighbourhood_cleansed True
neighbourhood_group_cleansed True
property_type True
room_type True
accommodates True
bathrooms True
bedrooms True
beds True
bed_type True
amenities True
square_feet False
review_scores_rating False
review_scores_accuracy True
review_scores_cleanliness True
review_scores_checkin True
review_scores_communication True
review_scores_location True
review_scores_value False
cancellation_policy True
calculated_host_listings_count True
price True
----------------
167
169
last_review for listing 1: 2015-12-29

last_review for listing 9: 2015-12-20

name for listing 1: Bright & Ai

These examples above are clearly duplicates. The only thing I see that's different is the review_scores_rating. This makes sense as these could have been scraped at different times and reviews could have been added, thus changing the rating. Clearly the same property though. My intuition is that using stuff that cannot change about a property, such as bedrooms, and bathrooms, room type (I would say someone listing the entire house vs. listing one room even if it's the same house constitutes a different listing), etc. is good to determine duplicates. But how many checks are required? Two different houses could both be 2bed/2bath, for example.  

In [18]:
subset_dropped_listings_street = new_seattle_listings.drop_duplicates(
    subset=['street']
)
print("Shape after dropping duplicates with exact match of street: ",
     subset_dropped_listings_street.shape)
subset_dropped_listings_sbd = new_seattle_listings.drop_duplicates(
    subset=['street', 'room_type', 'bedrooms', 'bathrooms']
)
print("Shape after dropping duplicates with exact match of street and checking \
for the same listing type within the same address: ",
     subset_dropped_listings_sbd.shape)

Shape after dropping duplicates with exact match of street:  (1442, 41)
Shape after dropping duplicates with exact match of street and checking for the same listing type within the same address:  (2621, 41)


I didn't think of this earlier but as seen here, there are a significant number of listings with the same street address. Removing those takes us from 3818 entries down to 1442 entries. However, it is possible that two listings could be on the same address but actually different properties or listing types. For example, For this simple check, I am happy to use this result, and just figure out how to keep the most recent listing with this result. In the new dataframe, I will also remove this street feature as it shouldn't influence the price.   

In [19]:
print("latest review for listing 1:", new_seattle_listings["last_review"][1])
print("latest review for listing 9:", new_seattle_listings["last_review"][9])

latest review for listing 1: 2015-12-29
latest review for listing 9: 2015-12-20


Just based on the above cell, comparing the last review dates for these two known duplicates (because the scrape date was the same for every entry so it doesn't provide any information), I will assume the most recent entries are at the top of the dataframe. Drop duplicates by default keeps the top entries so no modifications needed if proceeding based on this assumption. It's not time to assemble the actual dataframe based on 

In [20]:
#dropping out features that I am pretty sure will not be so useful or will be hard to 
#represent in a way that can train linear regression. I will maybe train another model that 
#attempts to represent features like description in a convoluted way and see if 
#it adds anything. Square feet is being dropped because almost all the values are missing
columns_to_drop = ["host_response_time", "host_since", "host_neighbourhood",
                   "neighbourhood_cleansed", "neighbourhood",
                   "last_review", "street", "name", "summary", "space", "description", 
                  "neighborhood_overview", "notes", "host_about", 
                   "host_total_listings_count","host_has_profile_pic", 
                   "square_feet", "calculated_host_listings_count"]
deduplicated_listing_df = subset_dropped_listings_sbd.drop(columns=columns_to_drop)
#re-indexing to make sure there are not gaps when we try to key by index
deduplicated_listing_df = deduplicated_listing_df.reset_index(drop=True)
print(deduplicated_listing_df.shape)

(2621, 23)


Now it's time to represent all the features numerically. Steps:
1) Represent categorical features (such as description or needs_verification) as a 
vector. This needs to be addressed on a feature by feature basis as well, for example use
one-hot encoding for something vs. a different method. As an example,
my thought for amenities is to find all the possible amenities, then have a binary feature for whether a house has each one. 
2) Impute any missing values for features that are already numerical.. Figure out feature 
by feature what the most logical way to do this is. It's possible that some entire 
examples will need to be removed.
3) Scaling and normalization.
4) I might need to remove some features because they either a) have too many missing values or b) are not feasible to easily represent as a number or a vector of numbers
5) Address outliers.


In [21]:
#Step 1: Convert categorial features to numbers
#Need to figure out what to do for each feature

print("List of features: ")
print(deduplicated_listing_df.columns.to_list())
print("-----------------")


List of features: 
['transit', 'host_response_rate', 'host_is_superhost', 'host_listings_count', 'host_identity_verified', 'neighbourhood_group_cleansed', 'property_type', 'room_type', 'accommodates', 'bathrooms', 'bedrooms', 'beds', 'bed_type', 'amenities', 'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value', 'cancellation_policy', 'price']
-----------------


Above, listing out the features again for convenience. We can print out all the possible values for transit to get a sense of the descriptions (below). Based on what I saw, the non-null descriptions all describe convenient access to some transit. Thus, I can assume non-null values have transit, and null values do not have transit. However, another approach I could try (I think I will do this) is finding some key words that relate to different forms of transit (bus, car, taxi/uber, walking, ferry, etc) and encode each of these as a binary variable (convenient_bus_access, convenient_car_access, etc)

In [22]:
#Getting a sense of transit
for idx in range(10):
    print("")
    print(deduplicated_listing_df['transit'][idx])


nan

Convenient bus stops are just down the block, if you're headed downtown or further afield to explore.  Zip cars are also strategically located on Queen Anne hill.

A bus stop is just 2 blocks away.   Easy bus access to all the sites: Lower Queen Anne, Seattle Center, The Space Needle, Experience Music Project, Chihuly Glass Museum, Pacific Science Center and more.

nan

The nearest public transit bus (D Line) is 2 blocks away next to the Whole Foods Market on 15th Ave. W. Downtown is a 15 minute bus ride or a 7 minute drive. The light rail located at Westlake Center in downtown runs to the airport. Parking: street parking is available - unlimited and no zoning

The #1 and #2 bus picks up 2-3 blocks away (different directions).  They both go downtown to Westlake.

The bus goes through the lower Queen Anne neighborhood past Seattle Center and to downtown.  From downtown you can catch the light rail to the airport or ride a ferry. It is harder to bike around our neighborhood because

In [23]:
#seeing how many listings have different types of transit close-by (based on popular keywords)
#to describe these forms of transportation
import re

keyword_map = {
    'bus': ['bus', 'line'],
    'car': ['car', 'drive', '99', '5', 'freeway', 'highway', 'hwy'],
    'ferry': ['ferry', 'boat'],
    'taxi': ['taxi', 'cab', 'Uber', 'Lyft'],
    'walk_bike': ['walk', 'bike', 'bicycle', 'foot'],
    'train': ['train', 'rail', 'subway', 'metro']
}

print("Overall number of listings:", deduplicated_listing_df.shape[0])
for transit_type, keywords in keyword_map.items():
    pattern = "|".join(re.escape(word) for word in keywords)
    count = deduplicated_listing_df['transit'].str.contains(
        pattern, case=False, na=False
    ).sum()
    print("Number of listings with convenient {} access: {}".format(transit_type, count))

Overall number of listings: 2621
Number of listings with convenient bus access: 1737
Number of listings with convenient car access: 1152
Number of listings with convenient ferry access: 61
Number of listings with convenient taxi access: 580
Number of listings with convenient walk_bike access: 911
Number of listings with convenient train access: 711


This seems reasonable. Now that we have a good sense of how many of each type of transit 
the properties have, let's convert these to binary numeric features. 

In [24]:
import re

#The keys to the dictionary are the new feature names we are inserting into the dataframe
keyword_map = {
    'convenient_bus_access': ['bus', 'line'],
    'convenient_car_access': ['car', 'drive', '99', '5', 'freeway', 'highway', 'hwy'],
    'convenient_ferry_access': ['ferry', 'boat'],
    'convenient_taxi_access': ['taxi', 'cab', 'Uber', 'Lyft'],
    'convenient_walk_bike_access': ['walk', 'bike', 'bicycle', 'foot'],
    'convenient_train_access': ['train', 'rail', 'subway', 'metro']
}

#Adding new binary features (the ones in the keyword map). For example 1 if transit 
#description contains one of the keywords, 0 otherwise. Done using regex 
for new_feature, keywords in keyword_map.items():
    pattern = "|".join(re.escape(word) for word in keywords)
    deduplicated_listing_df[new_feature] = deduplicated_listing_df['transit'].str.contains(
        pattern, case=False, na=False).astype(int)

#Dropping the transit column
df_transit_dropped = deduplicated_listing_df.drop(columns=['transit'])
print(df_transit_dropped.shape)

#Testing to see if the values match from the previous cell
for feature in keyword_map:
    print("Listings with {}: {}".format(feature, df_transit_dropped[feature].sum()))

#print(df_transit_dropped['transit']) produces a key error, as expected

(2621, 28)
Listings with convenient_bus_access: 1737
Listings with convenient_car_access: 1152
Listings with convenient_ferry_access: 61
Listings with convenient_taxi_access: 580
Listings with convenient_walk_bike_access: 911
Listings with convenient_train_access: 711


OK. Looks like the change to the dataframe was successful because the values match those from above and transit is now gone from the dataframe.

In [25]:
#Converting host response rate
print(df_transit_dropped.shape)
print("List of features: ")
print(df_transit_dropped.columns.to_list())
print("-------------------")
print(df_transit_dropped['host_response_rate'][:10])

df_response_rate = df_transit_dropped.copy()
df_response_rate['host_response_rate'] = (df_response_rate['host_response_rate']
                                          .str.strip('%')
                                          .astype(float)
                                          .fillna(0)
                                          .astype(int)
                                         )
print(df_response_rate.shape)
print(df_response_rate['host_response_rate'][:10])

(2621, 28)
List of features: 
['host_response_rate', 'host_is_superhost', 'host_listings_count', 'host_identity_verified', 'neighbourhood_group_cleansed', 'property_type', 'room_type', 'accommodates', 'bathrooms', 'bedrooms', 'beds', 'bed_type', 'amenities', 'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value', 'cancellation_policy', 'price', 'convenient_bus_access', 'convenient_car_access', 'convenient_ferry_access', 'convenient_taxi_access', 'convenient_walk_bike_access', 'convenient_train_access']
-------------------
0     96%
1     98%
2     67%
3     NaN
4    100%
5     NaN
6    100%
7    100%
8     NaN
9    100%
Name: host_response_rate, dtype: object
(2621, 28)
0     96
1     98
2     67
3      0
4    100
5      0
6    100
7    100
8      0
9    100
Name: host_response_rate, dtype: int64


In [26]:
print(df_response_rate['host_is_superhost'][:10])
df_tf = df_response_rate.copy()

#automatically detecting all columns that are just composed of true and false
tf_cols = [
    c for c in df_tf.columns
    if df_tf[c].dropna().isin(['t', 'f', 'T', 'F']).all()
]
#changing all true to 1 and false/NaN to 0
df_tf[tf_cols] = (
    df_tf[tf_cols]
    .apply(lambda col: col.str.strip().str.lower().eq('t'))
    .fillna(0)
    .astype(int)
)
print(df_tf['host_is_superhost'][:10])

0    f
1    t
2    f
3    f
4    f
5    f
6    t
7    t
8    f
9    t
Name: host_is_superhost, dtype: object
0    0
1    1
2    0
3    0
4    0
5    0
6    1
7    1
8    0
9    1
Name: host_is_superhost, dtype: int64


Above code takes care of all features that are made up of only true and false or NaN values. Not super valuable for this particular dataset because I think the only such feature is is_superhost, but still it seems like good practice.

In [27]:
#quick progress check on dataframe, will use as reference for feature engineering
feature_list = df_tf.columns.to_list()
for feature in feature_list:
    print(df_tf[feature].head(10))
    print("All unique values for {}: {}".format(feature, df_tf[feature].unique()))

0     96
1     98
2     67
3      0
4    100
5      0
6    100
7    100
8      0
9    100
Name: host_response_rate, dtype: int64
All unique values for host_response_rate: [ 96  98  67   0 100  71  97  60  50  31  90  70  88  63  33  80  99  83
  75  58  43  93  92  40  57  89  95  94  78  91  38  86  30  56  17  87
  64  82  69  65  25]
0    0
1    1
2    0
3    0
4    0
5    0
6    1
7    1
8    0
9    1
Name: host_is_superhost, dtype: int64
All unique values for host_is_superhost: [0 1]
0    3.0
1    6.0
2    2.0
3    1.0
4    2.0
5    1.0
6    1.0
7    5.0
8    1.0
9    1.0
Name: host_listings_count, dtype: float64
All unique values for host_listings_count: [  3.   6.   2.   1.   5.   9.   4.  36. 354.   8.  37.   7.  34.  10.
  13.  21.  11. 169. 502.  48.  12.  19.  nan  15.  84. 163.  18.]
0    1
1    1
2    1
3    1
4    1
5    1
6    1
7    1
8    1
9    1
Name: host_identity_verified, dtype: int64
All unique values for host_identity_verified: [1 0]
0    Queen Anne
1    Queen A

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


In [28]:
neighborhood_groups = df_tf['neighbourhood_group_cleansed'].unique().tolist()
#The keys to the dictionary are the new feature names we are inserting into the dataframe

ng_map = {
    "Is_In_{}".format(nh).replace(" ", "_"): nh for nh in neighborhood_groups
}

#Adding new binary features (the ones in the keyword map) by checking if regex is 
#exactly equal 
for new_feature in ng_map:
    pattern = ng_map[new_feature]
    df_tf[new_feature] = df_tf['neighbourhood_group_cleansed'].str.fullmatch(
        pattern, case=False, na=False).astype(int)


#Dropping the neighbourhood_group_cleansed column
df_ng_dropped = df_tf.drop(columns=['neighbourhood_group_cleansed'])
print(df_ng_dropped.shape)

#Testing to see if the values make sense
total = 0
for feature in ng_map:
    nh_total = df_ng_dropped[feature].sum()
    print(feature)
    print("Listings with {}: {}".format(feature, nh_total))
    total += nh_total
assert total == df_ng_dropped.shape[0], "Total number of properties \
in all neighborhoods doesn't match total number of properties"


(2621, 44)
Is_In_Queen_Anne
Listings with Is_In_Queen_Anne: 212
Is_In_Ballard
Listings with Is_In_Ballard: 177
Is_In_Other_neighborhoods
Listings with Is_In_Other_neighborhoods: 608
Is_In_Cascade
Listings with Is_In_Cascade: 58
Is_In_Central_Area
Listings with Is_In_Central_Area: 275
Is_In_University_District
Listings with Is_In_University_District: 65
Is_In_Downtown
Listings with Is_In_Downtown: 254
Is_In_Magnolia
Listings with Is_In_Magnolia: 56
Is_In_West_Seattle
Listings with Is_In_West_Seattle: 180
Is_In_Interbay
Listings with Is_In_Interbay: 9
Is_In_Beacon_Hill
Listings with Is_In_Beacon_Hill: 95
Is_In_Rainier_Valley
Listings with Is_In_Rainier_Valley: 122
Is_In_Delridge
Listings with Is_In_Delridge: 64
Is_In_Seward_Park
Listings with Is_In_Seward_Park: 33
Is_In_Northgate
Listings with Is_In_Northgate: 63
Is_In_Capitol_Hill
Listings with Is_In_Capitol_Hill: 296
Is_In_Lake_City
Listings with Is_In_Lake_City: 54


Now we do something similar for property type. However, unlike neighborhood groups, there are some property types that have very few examples, so those will just be grouped into Other along with any NaN values. The categories remaining will be (apartment, house, condominium, townhouse, loft, bed and breakfast, other).

In [29]:
property_types = df_ng_dropped['property_type'].unique()
print(property_types)
#The keys to the dictionary are the new feature names we are inserting into the dataframe
pt_map = { 
    "Is_{}".format(pt).replace(" ", "_"): {pt} for pt in property_types
}
pt_map["Is_Other"].update(["Cabin", "Bungalow", "Camper/RV", "Boat", "Dorm", 
                           "Treehouse", "Yurt", "Chalet", "Tent"])
entries_to_remove = {"Is_nan", "Is_Cabin", "Is_Bungalow", "Is_Camper/RV", "Is_Boat",
                    "Is_Dorm", "Is_Treehouse", "Is_Yurt", "Is_Chalet", "Is_Tent"}
for house_type in entries_to_remove:
    pt_map.pop(house_type, None)
print(pt_map)

#adding new binary features by using Series checks for whether they are in the 
#corresponding set
for new_feature in pt_map:
    #specifically including a logical or to make NaN entries go into "Is_Other" category
    if new_feature == "Is_Other":
        df_ng_dropped[new_feature] = df_ng_dropped['property_type'] \
            .isin(pt_map[new_feature]).astype(int) | df_ng_dropped['property_type'] \
            .isna().astype(int)
    else:
        df_ng_dropped[new_feature] = df_ng_dropped['property_type'] \
            .isin(pt_map[new_feature]).astype(int)

#Dropping the property_type column
df_pt_dropped = df_ng_dropped.drop(columns=['property_type'])
print(df_pt_dropped.shape)

#Testing to see if the values make sense
total = 0
for feature in pt_map:
    property_total = df_pt_dropped[feature].sum()
    print(feature)
    print("Listings with {}: {}".format(feature, property_total))
    total += property_total
assert total == df_pt_dropped.shape[0], "Total number of properties \
of all property types doesn't match total number of properties"

['Apartment' 'House' 'Cabin' 'Condominium' 'Bungalow' 'Townhouse'
 'Camper/RV' 'Loft' 'Boat' 'Bed & Breakfast' 'Other' 'Dorm' 'Treehouse'
 'Yurt' 'Chalet' 'Tent' nan]
{'Is_Apartment': {'Apartment'}, 'Is_House': {'House'}, 'Is_Condominium': {'Condominium'}, 'Is_Townhouse': {'Townhouse'}, 'Is_Loft': {'Loft'}, 'Is_Bed_&_Breakfast': {'Bed & Breakfast'}, 'Is_Other': {'Camper/RV', 'Dorm', 'Bungalow', 'Tent', 'Chalet', 'Yurt', 'Other', 'Cabin', 'Boat', 'Treehouse'}}
(2621, 50)
Is_Apartment
Listings with Is_Apartment: 1023
Is_House
Listings with Is_House: 1318
Is_Condominium
Listings with Is_Condominium: 58
Is_Townhouse
Listings with Is_Townhouse: 93
Is_Loft
Listings with Is_Loft: 31
Is_Bed_&_Breakfast
Listings with Is_Bed_&_Breakfast: 22
Is_Other
Listings with Is_Other: 76


Remaining "easy" conversions: room type, bed type, cancellation policy

More difficult: binary features for all possible amenities (the values of amenities themselves are sets, so we need to find the unique elements within those sets and combine ones like "Wi-Fi" and "Wifi")

In [30]:
#the same logic as property types but easier since we are just keeping the 
#3 categories as is without any processing

room_types = df_pt_dropped['room_type'].unique().tolist()
print(room_types)
rt_map = {
    "{}_Listed".format(rt).replace(" ", "_"): {rt} for rt in room_types
}
print(rt_map)
for new_feature in rt_map:
    pattern = rt_map[new_feature]
    df_pt_dropped[new_feature] = df_pt_dropped['room_type']\
        .isin(rt_map[new_feature]).astype(int)

#dropping the room type column since we don't need it
df_rt_dropped = df_pt_dropped.drop(columns=['room_type'])
print(df_rt_dropped.shape)

#testing if values make sense
total = 0
for feature in rt_map:
    room_type_total = df_rt_dropped[feature].sum()
    print(feature)
    print("Listings with {}: {}".format(feature, room_type_total))
    total += room_type_total
assert total == df_rt_dropped.shape[0], "Total number of properties of different room types does \
not match total number of properties"

['Entire home/apt', 'Private room', 'Shared room']
{'Entire_home/apt_Listed': {'Entire home/apt'}, 'Private_room_Listed': {'Private room'}, 'Shared_room_Listed': {'Shared room'}}
(2621, 52)
Entire_home/apt_Listed
Listings with Entire_home/apt_Listed: 1795
Private_room_Listed
Listings with Private_room_Listed: 754
Shared_room_Listed
Listings with Shared_room_Listed: 72


In [31]:
bed_types = df_pt_dropped['bed_type'].unique().tolist()
print(bed_types)
bt_map = {
    "Bed_Is_{}".format(bt).replace(" ", "_"):{bt} for bt in bed_types
}
#combining airbed and couch categories since they do not have a lot of entries
bt_map["Bed_Is_Airbed_Or_Couch"] = {"Airbed", "Couch"}
keys_to_remove = {"Bed_Is_Airbed", "Bed_Is_Couch"}
for key in keys_to_remove:
    bt_map.pop(key, None)
print(bt_map)

for new_feature in bt_map:
    df_pt_dropped[new_feature] = df_pt_dropped['bed_type']\
        .isin(bt_map[new_feature]).astype(int)
df_bt_dropped = df_pt_dropped.drop(columns=['bed_type'])
print(df_bt_dropped.shape)

total = 0
for feature in bt_map:
    bed_type_total = df_bt_dropped[feature].sum()
    print(feature)
    print("Number of listings where the {}: {}".format(feature, bed_type_total))
    total += bed_type_total
assert total == df_bt_dropped.shape[0], "Number of listings with all different bed types \
does not match total number of listings"

['Real Bed', 'Futon', 'Pull-out Sofa', 'Airbed', 'Couch']
{'Bed_Is_Real_Bed': {'Real Bed'}, 'Bed_Is_Futon': {'Futon'}, 'Bed_Is_Pull-out_Sofa': {'Pull-out Sofa'}, 'Bed_Is_Airbed_Or_Couch': {'Airbed', 'Couch'}}
(2621, 56)
Bed_Is_Real_Bed
Number of listings where the Bed_Is_Real_Bed: 2499
Bed_Is_Futon
Number of listings where the Bed_Is_Futon: 53
Bed_Is_Pull-out_Sofa
Number of listings where the Bed_Is_Pull-out_Sofa: 40
Bed_Is_Airbed_Or_Couch
Number of listings where the Bed_Is_Airbed_Or_Couch: 29


In [32]:
cancellation_policies = df_bt_dropped['cancellation_policy'].unique().tolist()
print(cancellation_policies)
cp_map = {
    "{}_cancellation_policy".format(cp):{cp} for cp in cancellation_policies
}
for new_feature in cp_map:
    df_bt_dropped[new_feature] = df_bt_dropped['cancellation_policy']\
        .isin(cp_map[new_feature]).astype(int)
df_cp_dropped = df_bt_dropped.drop(columns=['cancellation_policy'])
print(df_cp_dropped.shape)
total = 0
for feature in cp_map:
    cancellation_total = df_cp_dropped[feature].sum()
    print(feature)
    print("Number of listings with {}: {}".format(feature, cancellation_total))
    total += cancellation_total
assert total == df_cp_dropped.shape[0], "Number of cancellation policies does not equal \
total number of listings"

['moderate', 'strict', 'flexible']
(2621, 58)
moderate_cancellation_policy
Number of listings with moderate_cancellation_policy: 864
strict_cancellation_policy
Number of listings with strict_cancellation_policy: 929
flexible_cancellation_policy
Number of listings with flexible_cancellation_policy: 828


Now for the amenities. This requires a little bit of set processing.

In [33]:
amenities_sets = df_cp_dropped['amenities'].tolist()
print(amenities_sets[:10])
print("")
print(type(amenities_sets[0]))
print(len(amenities_sets))   #should be equal to df_cp_dropped.shape[0]

['{TV,"Cable TV",Internet,"Wireless Internet","Air Conditioning",Kitchen,Heating,"Family/Kid Friendly",Washer,Dryer}', '{TV,Internet,"Wireless Internet",Kitchen,"Free Parking on Premises","Buzzer/Wireless Intercom",Heating,"Family/Kid Friendly",Washer,Dryer,"Smoke Detector","Carbon Monoxide Detector","First Aid Kit","Safety Card","Fire Extinguisher",Essentials}', '{TV,"Cable TV",Internet,"Wireless Internet","Air Conditioning",Kitchen,"Free Parking on Premises","Pets Allowed","Pets live on this property",Dog(s),Cat(s),"Hot Tub","Indoor Fireplace",Heating,"Family/Kid Friendly",Washer,Dryer,"Smoke Detector","Carbon Monoxide Detector",Essentials,Shampoo}', '{Internet,"Wireless Internet",Kitchen,"Indoor Fireplace",Heating,"Family/Kid Friendly",Washer,Dryer,"Smoke Detector","Carbon Monoxide Detector","Safety Card","Fire Extinguisher",Essentials,Shampoo}', '{TV,"Cable TV",Internet,"Wireless Internet",Kitchen,Heating,"Family/Kid Friendly","Smoke Detector","Carbon Monoxide Detector","First Aid 

As we can see, printing out 10 of the amenities sets (I have also looked quickly at the whole list but I am just showing 10 here), we realize that each example of amenities is a string such as: '{TV, "Air Conditioner"}', where a set is enclosed inside the ' ', and some elements within the set have "" around them and some do not. We want to take the entire dataframe column and convert it into a single set of unique amenities that show up (for example {"TV", "Air Conditioner", "Wifi", ...}. We will use some regular expression magic courtesy of ChatGPT for this purpose

In [34]:
#'' on the outside of the regex means every element is wrapped by ''
#| is or, we have two options
#left: something wrapped by "". specifically 1 or more or anything that is not a double quote
#[^"]+ 
#right: 1 or more of alphanumeric, underscore, or spaces 

pattern = re.compile(r'"([^"]+)"|([A-Za-z0-9_ ]+)')
parsed_sets = [
    {a or b for a, b in pattern.findall(s)}
    for s in df_cp_dropped['amenities']
]
#* unpacks the set into individual arguments for union
unique_amenities = set().union(*parsed_sets)
#removing specifically 's' because we found it in there
unique_amenities.remove('s')
print(unique_amenities)
print("")
print("Number of unique amenities: {}".format(len(unique_amenities)))

{'Elevator in Building', 'Hot Tub', 'Wheelchair Accessible', 'Cable TV', 'Dryer', 'Heating', 'Cat', 'Safety Card', 'Iron', 'Essentials', 'Hangers', 'Washer', 'Smoke Detector', 'Free Parking on Premises', 'Air Conditioning', 'Pool', 'Pets Allowed', 'Shampoo', 'Wireless Internet', 'Fire Extinguisher', 'Washer / Dryer', 'Suitable for Events', 'Kitchen', 'Gym', 'First Aid Kit', 'Smoking Allowed', 'Breakfast', 'TV', 'Doorman', 'Buzzer/Wireless Intercom', 'Hair Dryer', 'Internet', '24-Hour Check-in', 'Dog', 'Other pet(s)', 'Carbon Monoxide Detector', 'Family/Kid Friendly', 'Laptop Friendly Workspace', 'Pets live on this property', 'Indoor Fireplace', 'Lock on Bedroom Door'}

Number of unique amenities: 41


41 unique amenities! That is a lot and very cumbersome to make features for. Let's see how many of each of these show up, and we can perhaps group some together to make features.

In [35]:
amenities_counter = Counter()
for am in parsed_sets:
    amenities_counter.update(am)
amenities_counter.pop("s")
print(amenities_counter)

Counter({'Wireless Internet': 2506, 'Heating': 2491, 'Kitchen': 2328, 'Smoke Detector': 2262, 'Essentials': 2209, 'Washer': 2043, 'Dryer': 2039, 'Internet': 1893, 'Shampoo': 1817, 'TV': 1776, 'Carbon Monoxide Detector': 1690, 'Free Parking on Premises': 1628, 'Fire Extinguisher': 1536, 'Family/Kid Friendly': 1423, 'First Aid Kit': 1181, 'Cable TV': 1025, 'Indoor Fireplace': 694, 'Pets live on this property': 663, 'Hangers': 528, 'Safety Card': 500, 'Hair Dryer': 497, 'Laptop Friendly Workspace': 487, 'Iron': 468, 'Air Conditioning': 427, 'Elevator in Building': 412, '24-Hour Check-in': 391, 'Dog': 379, 'Pets Allowed': 369, 'Cat': 286, 'Buzzer/Wireless Intercom': 282, 'Gym': 227, 'Hot Tub': 188, 'Breakfast': 188, 'Wheelchair Accessible': 178, 'Suitable for Events': 137, 'Pool': 74, 'Smoking Allowed': 57, 'Lock on Bedroom Door': 53, 'Other pet(s)': 41, 'Doorman': 39, 'Washer / Dryer': 2})


We can see above how many properties each amenity shows up in. Now we can start grouping at least some of these together. We kind of just need to do this manually I think. From this point, I will use sort of the same trick I had been using for the other features, even though there is probably a better way to do it. 

In [36]:
amenities_map = {
    "Has_{}".format(key).replace(" / ", "/").replace(" ", "_"):{key} \
        for key in amenities_counter
}

#Manually grouping features together. These are somewhat arbitrary, although I try to make
#the groupings make sense. 
#grouping amenities together
amenities_map["Has_Laundry"] = {"Washer", "Dryer", "Washer / Dryer", "Hangers", "Iron"}
    
amenities_map["Is_Pet_Friendly"] = {"Dog", "Cat", "Pets Allowed", "Other pet(s)",
                                   "Pets live on this property"}
amenities_map["Has_Internet"] = {"Internet", "Wireless Internet"}
amenities_map["Has_TV"] = {"TV", "Cable TV"}
amenities_map["Is_Wheelchair_Accessible"] = {"Wheelchair Accessible", "Elevator in Building"}
amenities_map["Has_Temperature_Control"] = {"Air Conditioning", "Heating"}
amenities_map["Has_Haircare"] = {"Shampoo", "Hair Dryer"}
amenities_map["Has_Some_Additional_Luxury"] = {"Hot Tub", "Indoor Fireplace", "Breakfast",
                                         "Suitable for Events", "Gym", "Pool", "Doorman"}

#removing all unnecessary elements from map after the grouping
elems_to_pop = {"Has_Washer/Dryer", "Has_Washer", "Has_Dryer", "Has_Hangers", "Has_Iron"}
pets_to_pop = {"Has_{}".format(pet).replace(" ", "_") for pet in amenities_map["Is_Pet_Friendly"]}
elems_to_pop.update(pets_to_pop)
elems_to_pop.update({"Has_Cable_TV", "Has_Wireless_Internet"})
elems_to_pop.update({"Has_Wheelchair_Accessible", "Has_Elevator_in_Building"})
elems_to_pop.update({"Has_Air_Conditioning", "Has_Heating"})
elems_to_pop.update({"Has_Shampoo", "Has_Hair_Dryer"})
elems_to_pop.update("Has_{}".format(luxury).replace(" ", "_") for luxury in amenities_map["Has_Some_Additional_Luxury"])
for elem in elems_to_pop:
    amenities_map.pop(elem, None)

#Checking to see what the updated map looks like
print(amenities_map)
print(len(amenities_map))

{'Has_Internet': {'Internet', 'Wireless Internet'}, 'Has_TV': {'TV', 'Cable TV'}, 'Has_Family/Kid_Friendly': {'Family/Kid Friendly'}, 'Has_Kitchen': {'Kitchen'}, 'Has_First_Aid_Kit': {'First Aid Kit'}, 'Has_Smoke_Detector': {'Smoke Detector'}, 'Has_Free_Parking_on_Premises': {'Free Parking on Premises'}, 'Has_Fire_Extinguisher': {'Fire Extinguisher'}, 'Has_Carbon_Monoxide_Detector': {'Carbon Monoxide Detector'}, 'Has_Safety_Card': {'Safety Card'}, 'Has_Essentials': {'Essentials'}, 'Has_Buzzer/Wireless_Intercom': {'Buzzer/Wireless Intercom'}, 'Has_24-Hour_Check-in': {'24-Hour Check-in'}, 'Has_Laptop_Friendly_Workspace': {'Laptop Friendly Workspace'}, 'Has_Lock_on_Bedroom_Door': {'Lock on Bedroom Door'}, 'Has_Smoking_Allowed': {'Smoking Allowed'}, 'Has_Laundry': {'Washer', 'Washer / Dryer', 'Iron', 'Dryer', 'Hangers'}, 'Is_Pet_Friendly': {'Cat', 'Dog', 'Pets live on this property', 'Pets Allowed', 'Other pet(s)'}, 'Is_Wheelchair_Accessible': {'Elevator in Building', 'Wheelchair Accessibl

In [37]:
#using the same regex from before to convert the strings in amenities to sets
pattern = re.compile(r'"([^"]+)"|([A-Za-z0-9_ ]+)')
df_cp_dropped['amenities_set'] = df_cp_dropped['amenities'].map(
    lambda s: {a or b for a, b in pattern.findall(s)}
)
#creating the binary columns using amenities_map
for new_feature, target_set in amenities_map.items():
    df_cp_dropped[new_feature] = df_cp_dropped['amenities_set'].map(lambda s, ts=target_set: int(bool(s & ts)))
#dropping unnecessary columns
df_amenities_dropped = df_cp_dropped.drop(columns=['amenities', 'amenities_set'])
print(df_amenities_dropped.shape)

(2621, 79)


That's it for feature converting! 79 features in all, a lot but later we can use correlation to see which features are important and cut it down. Now, it is time to impute missing values. First, we will get a sense of which columns have missing values, and how many. 

In [38]:
#Figure out what columns actually have missing values and how many. When we actually impute
#missing values, we will do it one feature at a time after transforming the feature to 
#numeric values if needed

missing_values = [feature for feature in df_amenities_dropped if 
                  df_amenities_dropped[feature].isna().any()]
#Find out how many missing values each column has
missing_value_counts = []
for feature in missing_values:
    missing_value_counts.append(df_amenities_dropped[feature].isna().sum())

#Convert to a map
missing_value_freq_map = {missing_values[i]: missing_value_counts[i] 
                          for i in range(len(missing_values))}
n_rows = df_amenities_dropped.shape[0]
print(n_rows)
missing_value_percent_map = {feature: round(missing_value_freq_map[feature] / n_rows, 3)
                        for feature in missing_value_freq_map}
print("Occurences of missing values")
print("------------------------------------")
print(missing_value_freq_map)
print("------------------------------------")
print(missing_value_percent_map)

2621
Occurences of missing values
------------------------------------
{'host_listings_count': 1, 'bathrooms': 12, 'bedrooms': 6, 'beds': 1, 'review_scores_rating': 464, 'review_scores_accuracy': 472, 'review_scores_cleanliness': 468, 'review_scores_checkin': 472, 'review_scores_communication': 468, 'review_scores_location': 471, 'review_scores_value': 472}
------------------------------------
{'host_listings_count': 0.0, 'bathrooms': 0.005, 'bedrooms': 0.002, 'beds': 0.0, 'review_scores_rating': 0.177, 'review_scores_accuracy': 0.18, 'review_scores_cleanliness': 0.179, 'review_scores_checkin': 0.18, 'review_scores_communication': 0.179, 'review_scores_location': 0.18, 'review_scores_value': 0.18}


In [39]:
df_imputed = df_amenities_dropped.copy()
price_replacements = {'$': '', ',':''}
df_imputed['price'] = df_amenities_dropped['price'].str.replace("$","").str \
    .replace(",","").astype(float)

#dropping room type since for some reason it didn't get dropped earlier
df_imputed = df_imputed.drop(columns=['room_type'])

#assuming that if listings count isn't listed, the current property is their only listing
df_imputed['host_listings_count'].fillna(1)

#imputing bedrooms/beds missing values based on the direct proportionality with the other 
mean_bedrooms = df_imputed['bedrooms'].mean()
mean_beds = df_imputed['beds'].mean()
ratio = mean_bedrooms / mean_beds
mask_bedrooms_missing = df_imputed['bedrooms'].isna() & df_imputed['beds'].notna()
df_imputed.loc[mask_bedrooms_missing, 'bedrooms'] = \
    round(df_imputed.loc[mask_bedrooms_missing, 'beds'] * ratio)
mask_beds_missing = df_imputed['beds'].isna() & df_imputed['bedrooms'].notna()
df_imputed.loc[mask_beds_missing, 'beds'] = \
    round(df_imputed.loc[mask_beds_missing, 'bedrooms'] / ratio)
mask_both_missing = df_imputed['beds'].isna() & df_imputed['bedrooms'].isna()
df_imputed.loc[mask_both_missing, 'beds'] = round(mean_beds)
df_imputed.loc[mask_both_missing, 'bedrooms'] = round(mean_bedrooms)

#filling the bathrooms in proportion to the bedrooms
mean_bathrooms = df_imputed['bathrooms'].mean()
mean_bedrooms = df_imputed['bedrooms'].mean()
bath_bed_ratio = mean_bathrooms / mean_bedrooms
df_imputed['bathrooms'] = df_imputed['bathrooms'] \
    .fillna(df_imputed['bedrooms'] * bath_bed_ratio)

#filling the rest of the columns with mean imputation
df_imputed = df_imputed.fillna(df_imputed.mean())
print(df_imputed.columns.tolist())
print("")
print("Checking if any columns have missing values:")
print(df_imputed.isna().any().tolist())

['host_response_rate', 'host_is_superhost', 'host_listings_count', 'host_identity_verified', 'accommodates', 'bathrooms', 'bedrooms', 'beds', 'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value', 'price', 'convenient_bus_access', 'convenient_car_access', 'convenient_ferry_access', 'convenient_taxi_access', 'convenient_walk_bike_access', 'convenient_train_access', 'Is_In_Queen_Anne', 'Is_In_Ballard', 'Is_In_Other_neighborhoods', 'Is_In_Cascade', 'Is_In_Central_Area', 'Is_In_University_District', 'Is_In_Downtown', 'Is_In_Magnolia', 'Is_In_West_Seattle', 'Is_In_Interbay', 'Is_In_Beacon_Hill', 'Is_In_Rainier_Valley', 'Is_In_Delridge', 'Is_In_Seward_Park', 'Is_In_Northgate', 'Is_In_Capitol_Hill', 'Is_In_Lake_City', 'Is_Apartment', 'Is_House', 'Is_Condominium', 'Is_Townhouse', 'Is_Loft', 'Is_Bed_&_Breakfast', 'Is_Other', 'Entire_home/apt_Listed', 'Private_room_Lis

As we can see, it looks like no columns have missing values anymore. 
Now, there are some features that might have a really high correlation with each other, so they are pretty much redundant. We should remove some of those and/or make composite features first. If this were a more sophisiticated project, we could use PCA. I will do that later.

In [40]:
correlation_matrix = df_imputed.corr()
columns = correlation_matrix.columns.tolist()
correlation_threshold = 0.7
high_corr = 0
for feature in columns:
    for j in range(correlation_matrix.shape[1]):
        if feature != columns[j] and \
        correlation_matrix[feature].iloc[j] >= correlation_threshold:
            print("Correlation between {} and {} is: {}."\
                  .format(feature, columns[j], correlation_matrix[feature].iloc[j]))
            high_corr += 1
print(f"Total pairs of features with high correlation between them: {high_corr}")

Correlation between accommodates and bedrooms is: 0.7993348693766243.
Correlation between accommodates and beds is: 0.8736630889631897.
Correlation between bedrooms and accommodates is: 0.7993348693766243.
Correlation between bedrooms and beds is: 0.7782120031949844.
Correlation between beds and accommodates is: 0.8736630889631897.
Correlation between beds and bedrooms is: 0.7782120031949844.
Total pairs of features with high correlation between them: 6


As we see from above, there are 3 pairs of feature that have high correlation (above my arbitrary threshold of 0.7). It says 6 but each of the 3 is actually double counted. The pairs are accomodates-bedrooms, accomodates-beds, and beds-bedrooms. For my purposes, this means I can remove two of the features and just use accomodates. 

In [41]:
df_imputed = df_imputed.drop(columns=['beds', 'bedrooms'])
print(df_imputed.shape)

(2621, 76)


Now, with 76 features that are not highly correlated to each other, that is a lot of features. I want to see which of the features are mostly correlated with price, and keep those to train the model.

In [42]:
correlation_matrix = df_imputed.corr()
columns = correlation_matrix.columns.tolist()

#for efficiency in the sorting later
idx_map = {feature:index for index, feature in enumerate(columns)}

price_index = idx_map['price']
order_importance = []
print("Correlation of different features with price.")
print("")
for i in range(correlation_matrix.shape[1]):
    print(f"{columns[i]}: {correlation_matrix['price'].iloc[i]}")
    if i != price_index:
        order_importance.append(columns[i])
#listing individual order of importance, not really used but just curiosity
#we cannot cut off least important ones from here, because there might be some interaction 
#with other features
order_importance.sort(key= lambda k: abs(correlation_matrix['price'].iloc[idx_map[k]]), reverse=True)
print("")
print("Order of importance (most to least): ")
print(order_importance)

Correlation of different features with price.

host_response_rate: -0.07831240186123826
host_is_superhost: -0.005302708124087684
host_listings_count: 0.0936484265670696
host_identity_verified: 0.0017747948970734356
accommodates: 0.6645972360914462
bathrooms: 0.5565111508677225
review_scores_rating: 0.040611573992094877
review_scores_accuracy: 0.005105436842849069
review_scores_cleanliness: 0.027781856656517848
review_scores_checkin: 0.019043301937000552
review_scores_communication: 0.018414361341174273
review_scores_location: 0.06348835796548899
review_scores_value: -0.049091588116863705
price: 1.0
convenient_bus_access: -0.05446096017793602
convenient_car_access: -0.07418973858017841
convenient_ferry_access: -0.01144813668897952
convenient_taxi_access: -0.018016336130601843
convenient_walk_bike_access: -0.02035984978587773
convenient_train_access: -0.0846900033661605
Is_In_Queen_Anne: 0.10349982763465118
Is_In_Ballard: -0.005643601309184372
Is_In_Other_neighborhoods: -0.04492875615574

Now, it is time to clip and normalize the listings. We will take a quick look at the DataFrame using describe and that will inform what type of normalization to use. We will also figure out what range to clip the features between, if needed.

In [43]:
pd.set_option('display.max_columns', None)
df_imputed.describe()
print(df_imputed.shape)

(2621, 76)


In [44]:
#can remove rest of review scores subcategories because I only now realized what they mean
#after seeing the numbers
#I will just use review_scores_rating (the composite rating) and hope the individual ones 
#of where those points are allocated don't matter much

rating_categories = ['accuracy', 'cleanliness', 'checkin', 'communication', 'location', 'value']
df_imputed = df_imputed.drop(columns=[f'review_scores_{c}' for c in rating_categories])
print(df_imputed.shape)

(2621, 70)


In [45]:
#choosing 90th percentile values to clip upper bound 
listings_count_q90 = df_imputed['host_listings_count'].quantile(0.90)
df_imputed['host_listings_count'] = df_imputed['host_listings_count'] \
    .clip(0, listings_count_q90)
accomodates_q90 = df_imputed['accommodates'].quantile(0.90)
df_imputed['accommodates'] = df_imputed['accommodates'].clip(0, accomodates_q90)
bathrooms_q90 = df_imputed['bathrooms'].quantile(0.90)
df_imputed['bathrooms'] = df_imputed['bathrooms'].clip(0, bathrooms_q90)
review_scores_rating_q10 = df_imputed['review_scores_rating'].quantile(0.10)
df_imputed['review_scores_rating'] = df_imputed['review_scores_rating'] \
    .clip(review_scores_rating_q10, 100)